In [73]:
import pandas as pd
import datetime as dt
tradebook_df = pd.read_csv('WAVE_1x_midcp.csv')
df = pd.read_csv("../data/midcp_wave.csv")
df["datetime"] = pd.to_datetime(df["time"])
df = df[df["datetime"].dt.year >= 2017]

INDEX = 'MIDCPNIFTY'
PORTFOLIO_VALUE = 10_00_000
# RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.0005
# LEVERAGE_ = 5

In [74]:
tradebook_df.rename(columns={'Unnamed: 0': 'Trade No.'}, inplace=True)
tradebook_df.head()


,Trade No.,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month
0,0,2022-01-10 15:15:00,LONG,2022-01-11 09:15:00,7739.45,7688.36963,7688.369630,2022-01-18 10:15:00,7928.05000,188.60000,180.766250,27114.937500,MACD Reversal,150,1,2.711494,2022,1
1,1,2022-01-27 14:15:00,LONG,2022-01-27 15:15:00,7360.35,7311.77169,7556.215000,2022-02-03 14:15:00,7624.95000,264.60000,257.107350,38566.102500,MACD Reversal,150,1,3.856610,2022,1
2,2,2022-02-09 10:15:00,LONG,2022-02-09 11:15:00,7476.70,7427.35378,7502.828125,2022-02-11 09:15:00,7439.55000,-37.15000,-44.608125,-6691.218750,TSL Hit,150,1,-0.669122,2022,2
3,3,2022-02-15 13:15:00,LONG,2022-02-15 14:15:00,7254.45,7206.57063,7206.570630,2022-02-17 14:15:00,7273.00000,18.55000,11.286275,1692.941250,MACD Reversal,150,1,0.169294,2022,2
4,4,2022-02-23 10:15:00,LONG,2022-02-23 11:15:00,7165.20,7117.90968,7117.909680,2022-02-23 13:15:00,7117.90968,-47.29032,-54.431875,-8164.781226,Initial SL Hit,150,1,-0.816478,2022,2


In [75]:
import pandas as pd

def calculate_day_by_day_pnl(df, tradebook_df):
    pnl_records = []
    
    for _, trade in tradebook_df.iterrows():
        trade_no = trade['Trade No.']
        start_date = pd.to_datetime(trade['Entry Time']).date()
        end_date = pd.to_datetime(trade['Exit Time']).date()
        # direction = trade['Trade_Type']  # 'long' or 'short'
        entry_price = trade['Entry Price']
        qty = trade['Qty']
        total_slippage = trade['Points Captured'] - trade['After Costs']
        
        # Filter the spot data for the trade duration
        trade_data = df[(df['datetime'].dt.date >= start_date) & (df['datetime'].dt.date <= end_date)].copy()
        
        # Aggregate data into daily OHLC
        trade_data['date'] = trade_data['datetime'].dt.date
        daily_ohlc = trade_data.groupby('date').agg(
            open_price=('open', 'first'),
            high_price=('high', 'max'),
            low_price=('low', 'min'),
            close_price=('close', 'last')
        ).reset_index()

        num_days = len(daily_ohlc)
        slippage_per_day = total_slippage / num_days if num_days > 0 else 0
        
        # Calculate day-by-day PnL
        for i, row in daily_ohlc.iterrows():
            date = row['date']
            exit_price = row['close_price']
            if i == 0:
                day_entry_price = entry_price
            else:
                day_entry_price = daily_ohlc.iloc[i - 1]['close_price']
            
            # if direction == 'Long':
            pnl = exit_price - day_entry_price
            # elif direction == 'short':
            #     pnl = day_entry_price - exit_price
            
            pnl_records.append({
                'Trade No.': trade_no,
                'Portfolio': PORTFOLIO_VALUE,
                'Date': date,
                'Entry Price': day_entry_price,
                'Exit Price': exit_price,
                'Points': pnl,
                'Qty': qty,
                'Slippage': slippage_per_day,
                'PnL': ((pnl - slippage_per_day) * qty),
                'ROI%': ((pnl - slippage_per_day) * qty) * 100 / PORTFOLIO_VALUE,
                'Index': INDEX,
            })
    
    # Convert the records into a DataFrame
    pnl_df = pd.DataFrame(pnl_records)
    return pnl_df


In [76]:
tb2 = calculate_day_by_day_pnl(df, tradebook_df)

In [77]:
tb2

,Trade No.,Portfolio,Date,Entry Price,Exit Price,Points,Qty,Slippage,PnL,ROI%,Index
0,0,1000000,2022-01-11,7739.45,7776.20,36.75,150,1.305625,5316.65625,0.531666,MIDCPNIFTY
1,0,1000000,2022-01-12,7776.20,7874.00,97.80,150,1.305625,14474.15625,1.447416,MIDCPNIFTY
2,0,1000000,2022-01-13,7874.00,7929.80,55.80,150,1.305625,8174.15625,0.817416,MIDCPNIFTY
3,0,1000000,2022-01-14,7929.80,7930.50,0.70,150,1.305625,-90.84375,-0.009084,MIDCPNIFTY
4,0,1000000,2022-01-17,7930.50,7944.30,13.80,150,1.305625,1874.15625,0.187416,MIDCPNIFTY
...,...,...,...,...,...,...,...,...,...,...,...
571,173,1000000,2024-11-22,12248.45,12309.75,61.30,100,2.479740,5882.02600,0.588203,MIDCPNIFTY
572,173,1000000,2024-11-25,12309.75,12590.65,280.90,100,2.479740,27842.02600,2.784203,MIDCPNIFTY
573,173,1000000,2024-11-26,12590.65,12564.00,-26.65,100,2.479740,-2912.97400,-0.291297,MIDCPNIFTY
574,173,1000000,2024-11-27,12564.00,12624.25,60.25,100,2.479740,5777.02600,0.577703,MIDCPNIFTY


In [78]:
# tb2.to_csv('JJPP_bnf_day-wise.csv', index=False)

In [79]:
tb2['Points'].sum()

4417.800000000004

In [80]:
(tradebook_df['Points Captured']).sum()

6083.306609999977

In [81]:
tb2['ROI%'].sum()

33.48938662062504

In [82]:
tb2.to_csv('midcp-daily-wave.csv', index=False)